In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from xgboost import XGBRegressor

# --------------------------
# 1️⃣ Generate Synthetic Data
# --------------------------

np.random.seed(42)

n_members = 2000
start_date = datetime(2015, 1, 1)
end_date = datetime(2025, 1, 1)

# Membership data
members = pd.DataFrame({
    "member_id": range(1, n_members + 1),
    "age": np.random.randint(20, 75, n_members),
    "gender": np.random.choice(["M", "F"], n_members),
    "plan_type": np.random.choice(["Silver", "Gold", "Platinum"], n_members, p=[0.5, 0.3, 0.2]),
    "join_date": [start_date + timedelta(days=np.random.randint(0, 365*5)) for _ in range(n_members)],
})

# Claim data
n_claims = 10000
claim_dates = [start_date + timedelta(days=np.random.randint(0, (end_date - start_date).days)) for _ in range(n_claims)]
claims = pd.DataFrame({
    "claim_id": range(1, n_claims + 1),
    "member_id": np.random.choice(members["member_id"], n_claims),
    "claim_date": claim_dates,
    "claim_amount": np.random.exponential(scale=15000, size=n_claims).astype(int)
})

# --------------------------
# 2️⃣ Define high-claim threshold
# --------------------------
threshold = claims["claim_amount"].quantile(0.9)
claims["is_high_claim"] = (claims["claim_amount"] > threshold).astype(int)

# --------------------------
# 3️⃣ Create historical (past 2 years) features & future (next 2 years) targets
# --------------------------

cutoff_date = datetime(2021, 1, 1)  # pretend we are training as of Jan 2021
past_window_start = cutoff_date - timedelta(days=365*2)
future_window_end = cutoff_date + timedelta(days=365*2)

# Past 2 years (feature window)
past_claims = claims[(claims["claim_date"] >= past_window_start) & (claims["claim_date"] < cutoff_date)]
# Future 2 years (target window)
future_claims = claims[(claims["claim_date"] >= cutoff_date) & (claims["claim_date"] < future_window_end)]

# Aggregate past claims
past_features = past_claims.groupby("member_id").agg(
    past_claim_count=("claim_id", "count"),
    past_total_amount=("claim_amount", "sum"),
    past_high_claims=("is_high_claim", "sum"),
    avg_claim_amount=("claim_amount", "mean"),
    last_claim_date=("claim_date", "max")
).reset_index()

# Fill missing (members with no past claims)
past_features["days_since_last_claim"] = (cutoff_date - past_features["last_claim_date"]).dt.days
past_features.drop(columns="last_claim_date", inplace=True)
past_features.fillna({"days_since_last_claim": 9999, "avg_claim_amount": 0,
                      "past_claim_count": 0, "past_total_amount": 0, "past_high_claims": 0}, inplace=True)

# Target: future high claim count & amount
future_targets = future_claims[future_claims["is_high_claim"] == 1].groupby("member_id").agg(
    future_high_claim_count=("claim_id", "count"),
    future_high_claim_amount=("claim_amount", "sum")
).reset_index()

# Merge features + targets + member info
data = members.merge(past_features, on="member_id", how="left").merge(future_targets, on="member_id", how="left")
data.fillna({"future_high_claim_count": 0, "future_high_claim_amount": 0}, inplace=True)

# --------------------------
# 4️⃣ Encode categorical variables
# --------------------------
data = pd.get_dummies(data, columns=["gender", "plan_type"], drop_first=True)

# --------------------------
# 5️⃣ Split Train/Test
# --------------------------
X = data.drop(columns=["member_id", "future_high_claim_count", "future_high_claim_amount", "join_date"])
y_count = data["future_high_claim_count"]
y_amount = data["future_high_claim_amount"]

X_train, X_test, y_count_train, y_count_test, y_amount_train, y_amount_test = train_test_split(
    X, y_count, y_amount, test_size=0.2, random_state=42
)

# --------------------------
# 6️⃣ Train Models
# --------------------------
count_model = XGBRegressor(objective='count:poisson', n_estimators=200, learning_rate=0.05, random_state=42)
count_model.fit(X_train, y_count_train)

amount_model = XGBRegressor(objective='reg:squarederror', n_estimators=200, learning_rate=0.05, random_state=42)
amount_model.fit(X_train, y_amount_train)

# --------------------------
# 7️⃣ Evaluate Models
# --------------------------
count_pred = count_model.predict(X_test)
amount_pred = amount_model.predict(X_test)

print("=== Claim COUNT Model ===")
print("MAE:", mean_absolute_error(y_count_test, count_pred))
print("R² :", r2_score(y_count_test, count_pred))

print("\n=== Claim AMOUNT Model ===")
print("MAE:", mean_absolute_error(y_amount_test, amount_pred))
print("R² :", r2_score(y_amount_test, amount_pred))

# --------------------------
# 8️⃣ Predict future for all members
# --------------------------
data["pred_future_high_claim_count"] = count_model.predict(X)
data["pred_future_high_claim_amount"] = amount_model.predict(X)
data["predicted_total_high_claim_cost"] = data["pred_future_high_claim_count"] * data["pred_future_high_claim_amount"]

# Display sample output
print("\nSample predictions:")
print(data[["member_id", "pred_future_high_claim_count", "pred_future_high_claim_amount", "predicted_total_high_claim_cost"]].head(10))


In [ ]:
# pmi_modeling.py
"""
Pipeline for EDA, feature engineering, XGBoost modeling, and SHAP explainability
on the provided PMI synthetic datasets.

Usage:
  1. Place `membership_10k_realistic.csv` and `claims_200k_realistic.csv`
     in the same folder as this script (or change DATA_DIR below).
  2. Install required packages (see README section below).
  3. Run: python pmi_modeling.py

Outputs are saved into a subfolder `model_outputs` inside DATA_DIR.
"""

import os
import warnings
import random
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score, accuracy_score, precision_score, recall_score, f1_score,
    mean_squared_error, mean_absolute_error, r2_score, confusion_matrix
)
import xgboost as xgb
import shap

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 200)
random.seed(42)
np.random.seed(42)

# -----------------------------
# Parameters / file locations
# -----------------------------
DATA_DIR = "."  # change to folder where CSVs are; e.g. "/path/to/csvs"
MEMBERSHIP_CSV = os.path.join(DATA_DIR, "membership_10k_realistic.csv")
CLAIMS_CSV = os.path.join(DATA_DIR, "claims_200k_realistic.csv")

# Prediction horizon: 2023 calendar year (modify if you want FY 2023-24)
PRED_START = pd.to_datetime("2023-01-01")
PRED_END   = pd.to_datetime("2023-12-31")

# Feature history cutoff: use history BEFORE PRED_START
HIST_CUTOFF = PRED_START - pd.Timedelta(days=1)

# High claim threshold (GBP) — change if you want different definition
HIGH_CLAIM_THRESHOLD = 10000.0

# Where to save outputs
OUT_DIR = os.path.join(DATA_DIR, "model_outputs")
os.makedirs(OUT_DIR, exist_ok=True)

RND = 42

# -----------------------------
# Utility functions
# -----------------------------
def safe_read_csv(path, parse_dates=None):
    print(f"Reading {path} ...")
    return pd.read_csv(path, parse_dates=parse_dates)

def print_shape(name, df):
    print(f"{name}: rows={len(df):,}, cols={df.shape[1]}")

# -----------------------------
# 1. LOAD DATA
# -----------------------------
membership = safe_read_csv(MEMBERSHIP_CSV, parse_dates=["Contract Start Date", "Contract End Date",
                                                       "Original Date of Joining", "Scheme Policy Joining Date", "Lapse Date"])
claims = safe_read_csv(CLAIMS_CSV, parse_dates=["Admission Date","Discharge Date","Incurred Date","Paid Date"])

print_shape("membership", membership)
print_shape("claims", claims)

# Normalize column names (strip)
membership.columns = [c.strip() for c in membership.columns]
claims.columns = [c.strip() for c in claims.columns]

# -----------------------------
# 2. PREPROCESS / CLEAN
# -----------------------------
assert "Unique Member Reference" in membership.columns
assert "Unique Member Reference" in claims.columns

for col in ["Contract Start Date","Contract End Date","Original Date of Joining","Scheme Policy Joining Date","Lapse Date"]:
    if col in membership.columns:
        membership[col] = pd.to_datetime(membership[col], errors="coerce")

for col in ["Admission Date","Discharge Date","Incurred Date","Paid Date"]:
    if col in claims.columns:
        claims[col] = pd.to_datetime(claims[col], errors="coerce")

# Keep only claims up to the end of prediction horizon (we only need history + target)
claims_hist = claims[claims["Admission Date"] <= PRED_END].copy()
print_shape("claims_hist (<= pred_end)", claims_hist)

# -----------------------------
# 3. BUILD TARGETS (prediction year)
# -----------------------------
mask_target = (claims_hist["Admission Date"] >= PRED_START) & (claims_hist["Admission Date"] <= PRED_END)
claims_target = claims_hist[mask_target].copy()
print_shape("claims_target (in prediction year)", claims_target)

claims_target["Claim Amount"] = pd.to_numeric(claims_target["Claim Amount"], errors="coerce").fillna(0.0)

target_agg = claims_target.groupby("Unique Member Reference")["Claim Amount"].agg(
    target_high_claim_count=lambda s: (s >= HIGH_CLAIM_THRESHOLD).sum(),
    target_high_claim_amount=lambda s: s[s >= HIGH_CLAIM_THRESHOLD].sum()
).reset_index()

members = membership[["Unique Member Reference"]].copy()
target_df = members.merge(target_agg, on="Unique Member Reference", how="left").fillna(0.0)
target_df["target_has_high_claim"] = (target_df["target_high_claim_count"] >= 1).astype(int)

print_shape("target_df", target_df)
target_df.to_csv(os.path.join(OUT_DIR, "target_summary_2023.csv"), index=False)

# -----------------------------
# 4. FEATURE ENGINEERING (use history before HIST_CUTOFF)
# -----------------------------
hist_claims = claims_hist[claims_hist["Admission Date"] <= HIST_CUTOFF].copy()
print_shape("hist_claims (<= hist_cutoff)", hist_claims)

hist_claims["Claim Amount"] = pd.to_numeric(hist_claims["Claim Amount"], errors="coerce").fillna(0.0)
hist_claims["Amount Paid"] = pd.to_numeric(hist_claims.get("Amount Paid", pd.Series(0, index=hist_claims.index)), errors="coerce").fillna(0.0)

agg_funcs = {
    "Claim Amount": ["count", "sum", "mean", "median", "max"],
    "Amount Paid": ["sum","mean"]
}
hist_agg = hist_claims.groupby("Unique Member Reference").agg(agg_funcs)
hist_agg.columns = ["_".join(col).strip() for col in hist_agg.columns.values]
hist_agg = hist_agg.reset_index().rename(columns={
    "Claim Amount_count": "hist_claim_count",
    "Claim Amount_sum": "hist_claim_amount_sum",
    "Claim Amount_mean": "hist_claim_amount_mean",
    "Claim Amount_median": "hist_claim_amount_median",
    "Claim Amount_max": "hist_claim_amount_max",
    "Amount Paid_sum": "hist_paid_sum",
    "Amount Paid_mean": "hist_paid_mean"
})

hist_high = hist_claims[hist_claims["Claim Amount"] >= HIGH_CLAIM_THRESHOLD].groupby("Unique Member Reference")["Claim Amount"].agg(
    hist_high_claim_count="count", hist_high_claim_amount_sum="sum"
).reset_index()

# Condition counts pivot (top N conditions)
top_conditions = hist_claims["Condition Category"].value_counts().nlargest(15).index.tolist()
cond_counts = hist_claims.groupby(["Unique Member Reference","Condition Category"]).size().unstack(fill_value=0)
for c in top_conditions:
    if c not in cond_counts.columns:
        cond_counts[c] = 0
cond_counts_reduced = cond_counts[top_conditions].reset_index().rename(columns={c: f"cond_cnt_{c}" for c in top_conditions})

# Claim type counts
ctype_counts = hist_claims.groupby(["Unique Member Reference","Claim Type"]).size().unstack(fill_value=0).reset_index()
ctype_counts.columns = ["Unique Member Reference"] + [f"ctype_cnt_{c}" for c in ctype_counts.columns[1:]]

# Recent activity: days since last claim before HIST_CUTOFF
last_claim = hist_claims.groupby("Unique Member Reference")["Admission Date"].max().reset_index()
last_claim["days_since_last_claim"] = (HIST_CUTOFF - last_claim["Admission Date"]).dt.days
last_claim = last_claim[["Unique Member Reference","days_since_last_claim"]]

# Merge features
features = members.merge(hist_agg, on="Unique Member Reference", how="left")
features = features.merge(hist_high, on="Unique Member Reference", how="left")
features = features.merge(cond_counts_reduced, on="Unique Member Reference", how="left")
features = features.merge(ctype_counts, on="Unique Member Reference", how="left")
features = features.merge(last_claim, on="Unique Member Reference", how="left")
features.fillna(0, inplace=True)

# Add member fields (demographics / scheme / client)
member_cols_to_use = [
    "Unique Member Reference","Client Identifier","Client Name","Scheme Category/Section Name",
    "Year of Birth","Gender","Short Post Code of Member","Status of Member","Contract Start Date","Contract End Date"
]
meta = membership[member_cols_to_use].copy()
meta["age_at_cutoff"] = HIST_CUTOFF.astype('datetime64[Y]').astype(int) - meta["Year of Birth"]
meta["contract_start"] = pd.to_datetime(meta["Contract Start Date"], errors="coerce")
meta["tenure_days"] = (HIST_CUTOFF - meta["contract_start"]).dt.days.clip(lower=0).fillna(0)

# merge meta
features = features.merge(meta.drop(columns=["Contract Start Date","Contract End Date"]), on="Unique Member Reference", how="left")
features.fillna(0, inplace=True)

# -----------------------------
# 5. MERGE TARGET & FINALIZE FEATURE MATRIX
# -----------------------------
df = features.merge(target_df, on="Unique Member Reference", how="left")
df.fillna(0, inplace=True)
print_shape("model dataframe", df)
df.to_csv(os.path.join(OUT_DIR, "features_with_target.csv"), index=False)

# -----------------------------
# 6. SIMPLE EDA & CORRELATION
# -----------------------------
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
corr = df[numeric_cols].corr()
corr.to_csv(os.path.join(OUT_DIR, "correlation_matrix.csv"))

plt.figure(figsize=(10,8))
sns.heatmap(corr.abs(), cmap='viridis', cbar=True)
plt.title("Absolute Correlation Matrix (numeric features)")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "corr_matrix.png"), dpi=150)
plt.close()

# -----------------------------
# 7. PREPARE DATA FOR MODELING
# -----------------------------
y = df["target_has_high_claim"].astype(int)
y_reg = df["target_high_claim_amount"].astype(float)

drop_cols = [
    "Unique Member Reference","Client Name","Client Identifier","Scheme Category/Section Name",
    "target_high_claim_count","target_high_claim_amount","target_has_high_claim"
]
X = df.drop(columns=[c for c in drop_cols if c in df.columns])

# Extract postcode area (outward) and encode basic demographics
def extract_postcode_area(pc):
    try:
        return str(pc).split(" ")[0]
    except:
        return "UNK"

X["postcode_area"] = X["Short Post Code of Member"].fillna("UNK").apply(extract_postcode_area)
X["gender_num"] = X["Gender"].map({"M":0,"F":1,"B":2}).fillna(-1)

# Frequency encode top clients
top_clients = X["Client Identifier"].value_counts().nlargest(20).index.tolist()
X["client_top"] = X["Client Identifier"].apply(lambda x: x if x in top_clients else "OTHER_CLIENT")
X["scheme_cat"] = X["Scheme Category/Section Name"].astype(str).fillna("UNK")

# Drop raw columns and one-hot small cardinality columns
for c in ["Short Post Code of Member","Gender","Client Identifier","Scheme Category/Section Name"]:
    if c in X.columns:
        X.drop(columns=[c], inplace=True)

ohe_cols = ["client_top","scheme_cat","postcode_area"]
X = pd.get_dummies(X, columns=ohe_cols, drop_first=True, dummy_na=False)
X.fillna(0, inplace=True)

print("Final feature matrix shape:", X.shape)

# Train/test split with stratify on rare target
X_train, X_test, y_train, y_test, yreg_train, yreg_test = train_test_split(
    X, y, y_reg, test_size=0.2, random_state=RND, stratify=y
)
print("Train/test sizes:", X_train.shape, X_test.shape)

# -----------------------------
# 8. CLASSIFIER: XGBoost
# -----------------------------
clf = xgb.XGBClassifier(
    n_estimators=300, max_depth=6, learning_rate=0.05, subsample=0.8,
    colsample_bytree=0.8, random_state=RND, use_label_encoder=False, eval_metric="logloss"
)

clf.fit(X_train, y_train, eval_set=[(X_train,y_train),(X_test,y_test)], verbose=50, early_stopping_rounds=30)

# Predictions & metrics
y_pred_proba = clf.predict_proba(X_test)[:,1]
y_pred = (y_pred_proba >= 0.5).astype(int)

roc = roc_auc_score(y_test, y_pred_proba)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, zero_division=0)
rec = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
print(f"Classifier ROC AUC: {roc:.4f}, Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}, F1: {f1:.4f}")

cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)

clf.save_model(os.path.join(OUT_DIR, "xgb_classifier.model"))

fi = pd.DataFrame({
    "feature": X.columns,
    "importance": clf.feature_importances_
}).sort_values("importance", ascending=False)
fi.to_csv(os.path.join(OUT_DIR, "clf_feature_importance.csv"), index=False)

# -----------------------------
# 9. SHAP EXPLAINABILITY (classifier)
# -----------------------------
explainer = shap.TreeExplainer(clf)
X_test_small = X_test.sample(n=min(2000, len(X_test)), random_state=RND)
shap_values = explainer.shap_values(X_test_small)

shap.summary_plot(shap_values, X_test_small, show=False)
plt.title("SHAP summary (classifier)")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "shap_summary_classifier.png"), dpi=150)
plt.close()

top_feat = fi.iloc[0]["feature"]
try:
    shap.dependence_plot(top_feat, shap_values, X_test_small, show=False)
    plt.savefig(os.path.join(OUT_DIR, f"shap_dependence_{top_feat}.png"), dpi=150)
    plt.close()
except Exception as e:
    print("SHAP dependence plot failed for feature:", top_feat, e)

# Save SHAP values sample and corresponding X sample
shap_df = pd.DataFrame(shap_values, columns=X_test_small.columns)
shap_df.to_csv(os.path.join(OUT_DIR, "shap_values_sample.csv"), index=False)
X_test_small.reset_index(drop=True).to_csv(os.path.join(OUT_DIR, "X_test_shap_sample.csv"), index=False)

# -----------------------------
# 10. REGRESSOR: XGBoost for high-claim amount
# -----------------------------
reg = xgb.XGBRegressor(
    n_estimators=400, max_depth=6, learning_rate=0.05, subsample=0.8,
    colsample_bytree=0.8, random_state=RND
)

reg.fit(X_train, yreg_train, eval_set=[(X_train,yreg_train),(X_test,yreg_test)], verbose=50, early_stopping_rounds=30)

yreg_pred = reg.predict(X_test)
mse = mean_squared_error(yreg_test, yreg_pred)
mae = mean_absolute_error(yreg_test, yreg_pred)
r2 = r2_score(yreg_test, yreg_pred)
print(f"Regressor MSE={mse:.2f}, MAE={mae:.2f}, R2={r2:.4f}")

reg.save_model(os.path.join(OUT_DIR, "xgb_regressor.model"))
fi_reg = pd.DataFrame({"feature": X.columns, "importance": reg.feature_importances_}).sort_values("importance", ascending=False)
fi_reg.to_csv(os.path.join(OUT_DIR, "reg_feature_importance.csv"), index=False)

explainer_reg = shap.TreeExplainer(reg)
X_test_small_reg = X_test.sample(n=min(2000,len(X_test)), random_state=RND)
shap_values_reg = explainer_reg.shap_values(X_test_small_reg)

shap.summary_plot(shap_values_reg, X_test_small_reg, show=False)
plt.title("SHAP summary (regressor)")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "shap_summary_regressor.png"), dpi=150)
plt.close()

# -----------------------------
# 11. OUTPUTS & REPORT ASSETS
# -----------------------------
test_results = X_test.copy()
test_results["y_true_has_high"] = y_test.values
test_results["y_pred_proba_has_high"] = y_pred_proba
test_results["y_pred_has_high"] = y_pred
test_results["y_true_high_amount"] = yreg_test.values
test_results["y_pred_high_amount"] = yreg_pred
test_results.to_csv(os.path.join(OUT_DIR, "test_predictions.csv"), index=False)

with open(os.path.join(OUT_DIR, "model_metrics.txt"), "w") as f:
    f.write("Classifier metrics:\n")
    f.write(f"ROC AUC: {roc:.4f}\nAccuracy: {acc:.4f}\nPrecision: {prec:.4f}\nRecall: {rec:.4f}\nF1: {f1:.4f}\n\n")
    f.write("Regressor metrics:\n")
    f.write(f"MSE: {mse:.2f}\nMAE: {mae:.2f}\nR2: {r2:.4f}\n")

print("All outputs saved to:", OUT_DIR)
